In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install datasets==2.16.1

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_

In [3]:
from datasets import load_dataset, DatasetDict

ds_train = load_dataset("huggingface-course/codeparrot-ds-train", split="train")
ds_valid = load_dataset("huggingface-course/codeparrot-ds-valid", split="validation")

raw_datasets = DatasetDict(
    {
        "train": ds_train,  # .shuffle().select(range(50000)),
        "valid": ds_valid,  # .shuffle().select(range(500))
    }
)

raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 606720
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 3322
    })
})

In [23]:
raw_datasets = DatasetDict(
    {
        "train": ds_train.shuffle().select(range(356720)),
        "valid": ds_valid.shuffle().select(range(500))
    }
)

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 356720
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 500
    })
})

In [24]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('huggingface-course/code-search-net-tokenizer')

In [25]:
raw_datasets = raw_datasets.map(
    lambda x:tokenizer(x['content'],return_overflowing_tokens=True,return_length=True,truncation=True,max_length = 128)
    ,batched = True,remove_columns=raw_datasets['train'].column_names)
raw_datasets

Map:   0%|          | 0/356720 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
raw_datasets = raw_datasets.filter(lambda x:x['length'] == 128)
raw_datasets

In [ ]:
from transformers import AutoConfig, GPT2LMHeadModel

config = AutoConfig.from_pretrained(
    'gpt2',
    vocab_size = len(tokenizer),
    n_ctx = 128,
    bos_token_id = tokenizer.bos_token_id,
    eos_token_id = tokenizer.eos_token_id

)

In [ ]:
model = GPT2LMHeadModel(config)

In [ ]:
print(f'{model.num_parameters():,}')

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer,mlm=False)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
data_collator([raw_datasets['train'][i] for i in range(2)])

In [ ]:
from transformers import TrainingArguments, Trainer



args = TrainingArguments('TextGeneration-Coding-Data-Pretrained',
                                   per_device_train_batch_size = 32,
                                   per_device_eval_batch_size = 32,
                                   evaluation_strategy = 'steps',
                                   eval_steps = 5000,
                                   logging_steps = 5000,
                                   gradient_accumulation_steps = 8,
                                   num_train_epochs=1,
                                   weight_decay = 0.1,
                                   warmup_steps = 1000,
                                   lr_scheduler_type = 'cosine',
                                   learning_rate = 5e-4,
                                   save_steps = 5000,
                                   fp16=True,
                                   push_to_hub=True)


trainer = Trainer(model = model,
                  tokenizer = tokenizer,
                  args = args,
                  data_collator = data_collator,
                  train_dataset = raw_datasets['train'],
                  eval_dataset = raw_datasets['valid'])



In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
import torch
from transformers import pipeline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline(
    "text-generation", model="HamzaNaser/TextGeneration-Coding-Data-Pretrained", device=device
)

In [ ]:
text = """
"""

print(pipe(text)[0]['generated_text'])

In [ ]:
text = """import numpy as np
# Create a function to add arr1 and arr2 together
def sum_arrays(arr1,arr2):
  return
"""

print(pipe(text)[0]['generated_text'])